In [ ]:
import zipfile

In [ ]:
sourcefolder = '../../../../Downloads'

f = ['PRECIP-COLUMN', 'SNOW-PROFILE']
#year_month = 201611
#t = ['2016317111800_56087',
 #   '2016318102222_56101',
  #  '2016321105315_56145',
   # '2016322095737_56159',
    #'2016323104052_56174',
    #'2016334102212_56334']
#year_month =  201612
#t = ['2016356112357_56655', 
 #     '2016357102821_56669',
  #    '2016358111138_56684',
   #   '2016359101602_56698',
    #  '2016360105918_56713',
     # '2016361100342_56727']
year_month = 201701
t = ['2017003105316_56844',
    '2017006112413_56888',
    '2017009101617_56931',
    '2017010105933_56946',
    '2017011100357_56960',
    '2017012104713_56975',
    '2017013095137_56989',
     '2017029095148_57222',
    '2017029113041_57223']
#year_month = 201702
#t = ['2017033110600_57281',
 #   '2017034101023_57295',
  #  '2017035105339_57310']

targetfolder = '../../Data/CloudSat/2C_Precip/%s/' %year_month
savefig = 0

https://www.science-emergence.com/Articles/How-to-read-a-MODIS-HDF-file-using-python-/

https://www.science-emergence.com/Articles/How-to-read-CloudSat-2B-GEOPROF-GRANULE-HDF4-file-using-python-and-pyhdf-/

In [ ]:
import sys
sys.path.append('/uio/kant/geo-metos-u7/franzihe/Documents/Thesis/Python')
import createFolder as cF
### Import python libraries
from pyhdf.SD import SD, SDC
from pyhdf.HDF import *
from pyhdf.VS import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from mpl_toolkits.basemap import shiftgrid


import pprint

In [ ]:
stn_lat      = 59.81
stn_lon      = 7.21

lower_lat = stn_lat -2.
upper_lat = stn_lat +2.

left_lon = stn_lon -2.
right_lon = stn_lon +2.


http://www.trondkristiansen.com/?page_id=846

Since the ETOPO1 file is quite large (445MB) I added a function to the script that cuts out only the area of interest from the ETOPO1 file. This saves space and time (see function: findSubsetIndices).

In [ ]:
def findSubsetIndices(min_lat,max_lat,min_lon,max_lon,lats,lons):
    
    """Array to store the results returned from the function"""
    res=np.zeros((4),dtype=np.float64)
    minLon=min_lon; maxLon=max_lon
    
    distances1 = []; distances2 = []
    indices=[]; index=1
    
    for point in lats:
        s1 = max_lat-point # (vector subtract)
        s2 = min_lat-point # (vector subtract)
        distances1.append((np.dot(s1, s1), point, index))
        distances2.append((np.dot(s2, s2), point, index-1))
        index=index+1
        
    distances1.sort()
    distances2.sort()
    indices.append(distances1[0])
    indices.append(distances2[0])
    
    distances1 = []; distances2 = []; index=1
   
    for point in lons:
        s1 = maxLon-point # (vector subtract)
        s2 = minLon-point # (vector subtract)
        distances1.append((np.dot(s1, s1), point, index))
        distances2.append((np.dot(s2, s2), point, index-1))
        index=index+1
        
    distances1.sort()
    distances2.sort()
    indices.append(distances1[0])
    indices.append(distances2[0])
    
    """ Save final product: max_lat_indices,min_lat_indices,max_lon_indices,min_lon_indices"""
    minJ=indices[1][2] # min_lat_indices
    maxJ=indices[0][2]
    minI=indices[3][2]
    maxI=indices[2][2] # max_lon_indices
    
    res[0]=minI; res[1]=maxI; res[2]=minJ; res[3]=maxJ;
    return res

In [ ]:
for file in f:
    if file == 'PRECIP-COLUMN':
        processing = 'P2'
    elif file == 'SNOW-PROFILE':
        processing = 'P'
for day in t:
    file_path = targetfolder
    file_name = '/%s_CS_2C-%s_GRANULE_%s_R04_E06.hdf' %(day,file,processing)

### Read latitudes and longitudes, profile time
    f = HDF(file_path+file_name, SDC.READ) 
    vs = f.vstart()

    data_info_list = vs.vdatainfo()

#pprint.pprint( data_info_list )

    vdata_lat = vs.attach('Latitude')
    vdata_long = vs.attach('Longitude')

    vdata_profile_time = vs.attach('Profile_time')
    vdata_snowfall_rate_sfc = vs.attach('snowfall_rate_sfc')



    lat = vdata_lat[:]
    long = vdata_long[:]

    profile_time = vdata_profile_time[:]
    snowfall_rate_sfc = vdata_snowfall_rate_sfc[:]


#print('Nb pixels: ', len(lat))
#print('Lat min, Lat max: ',min(lat),max(lat))

#for i in range(15): # sample
 #   print(lat[i])

# to get vdata latitude info
#print( vdata_lat.attrinfo() )        
        
    vdata_lat.detach() # "close" the vdata
    vdata_long.detach() # "close" the vdata
    vdata_profile_time.detach()
    vdata_snowfall_rate_sfc.detach()
    vs.end() # terminate the vdata interface
    f.close()




# Create a basemap to show the data
    m = Basemap(projection='cyl',llcrnrlat=lower_lat-2,urcrnrlat=upper_lat+2, 
            llcrnrlon=left_lon-2,urcrnrlon=right_lon+2,resolution='c')
    m.drawcoastlines()
    m.drawparallels(np.arange(-90.,90.,30.),labels=[False,True,True,False])
    m.drawmeridians(np.arange(-180.,180.,30.),labels=[True,False,False,True])
    
    x,y = m(long,lat)


    m.scatter(x,y,3,marker='o',color='b')
    m.scatter(stn_lon,stn_lat,5, marker='x')
    plt.title("Cloudsat Trajectory %s" %day)

    if savefig == 1:
        fig_dir = '../../Figures/cloudsat_trajectory/%s/' %(year_month)
        cF.createFolder(fig_dir)
        plt.savefig(fig_dir+"%s_2deg.png"  %day, bbox_inches='tight', dpi=200)

    plt.show()
    plt.close()

#    res = findSubsetIndices(lower_lat,upper_lat,left_lon,right_lon,np.array(lat),np.array(long))

 #   lons, lats = np.meshgrid(long,lat)

  #  data, la = np.meshgrid(snowfall_rate_sfc, lat)